### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import time
from glob import glob
from tqdm import tqdm
from scipy import interpolate

In [3]:
# conda install tensorflow

In [4]:
# conda install python=3.7

In [5]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

CNN으로 전처리해볼 것
1. 유량 3개, 수위 4개
2. 팔당댐 데이터

### 전체 수위 데이터셋 구축

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 데이터 불러오기 
전체df = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
# for colab
# /content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv
# for jupyter
# /Volumes/GoogleDrive/.shortcut-targets-by-id/1rUx7_7LmBqAQTbWEWVTGvw0_d0v6N2ZT/신입기수 프로젝트/Dataset/rawdata.csv

In [8]:
수위 = 전체df[['청담대교_수위', '한강대교_수위', '잠수교_수위', '행주대교_수위']]

In [9]:
수위

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
276331,0.0,0.0,0.0,0.0
276332,0.0,0.0,0.0,0.0
276333,0.0,0.0,0.0,0.0
276334,0.0,0.0,0.0,0.0


In [10]:
nanIndex = 수위[수위.청담대교_수위.isnull()].index

In [11]:
nanIndex

Int64Index([140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620,
            140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628,
            140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636,
            140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644,
            140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652,
            140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660,
            140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668,
            140669, 140670, 236080],
           dtype='int64')

In [12]:
consecutive = list(nanIndex)[:-1]
unconsecutive = list(nanIndex)[-1]
print(consecutive)
print(unconsecutive)

[140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
236080


In [13]:
# 58개 결측치 앞 인덱스까지 인풋 생성
x = 수위.loc[:140612]

# 결측치로 아웃풋 생성
null = 수위[140613:140671]

In [14]:
null

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
140613,NaN,NaN,NaN,NaN
140614,NaN,NaN,NaN,NaN
140615,NaN,NaN,NaN,NaN
140616,NaN,NaN,NaN,NaN
140617,NaN,NaN,NaN,NaN
140618,NaN,NaN,NaN,NaN
140619,NaN,NaN,NaN,NaN
140620,NaN,NaN,NaN,NaN
140621,NaN,NaN,NaN,NaN
140622,NaN,NaN,NaN,NaN


In [15]:
x

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,310.7,290.0,300.2,275.3
1,314.7,290.0,300.2,275.3
2,313.7,290.0,301.2,275.3
3,311.7,290.0,301.2,276.3
4,311.7,291.0,301.2,277.3
...,...,...,...,...
140608,435.7,410.0,415.2,374.3
140609,438.7,413.0,418.2,375.3
140610,437.7,413.0,418.2,373.3
140611,434.7,413.0,415.2,373.3


In [16]:
# conda install scikit-learn

In [17]:
# train, test data 생성할 때 필요
from sklearn.model_selection import train_test_split
# 정규화할 때 필요
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [18]:
# input 정규화
x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = StandardScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    x_regular.iloc[:,i] = tmp_x

In [19]:
x_regular

,청담대교_수위,한강대교_수위,잠수교_수위,행주대교_수위
0,-0.213183,-0.292141,-0.168958,-0.473340
1,-0.150481,-0.292141,-0.168958,-0.473340
2,-0.166156,-0.292141,-0.151149,-0.473340
3,-0.197507,-0.292141,-0.151149,-0.450834
4,-0.197507,-0.273710,-0.151149,-0.428328
...,...,...,...,...
140608,1.746252,1.919546,1.879052,1.754767
140609,1.793278,1.974838,1.932478,1.777274
140610,1.777603,1.974838,1.932478,1.732261
140611,1.730576,1.974838,1.879052,1.732261


### 잠수교 수위

In [20]:
start = time.time()

y_predict = []

xtrain = []
ytrain = []

w =  432 # window size
n = 58 # 예측할 데이터 개수

# xtrain 생성
# for i in range(len(x_regular)-w):
#     xtrain.append(np.array(x_regular.iloc[i:i + w, :]).astype(float))

for i in tqdm(range(len(x_regular)-w-n)):
    xtrain.append(np.array(x_regular.iloc[i:i + w, 2]).astype(float))

100%|██████████| 140123/140123 [00:14<00:00, 9672.32it/s] 


In [21]:
# ytrain 생성
# for i in range(w, len(x_regular)):
#     ytrain.append(np.array(x_regular.iloc[i, :]).astype(float))
ytrain = []

for i in tqdm(range(w, len(x_regular)-n)):
  ytrain.append(np.array(x_regular.iloc[i:i+n, 2]).astype(float))

100%|██████████| 140123/140123 [00:14<00:00, 9717.49it/s] 


In [22]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
print(xtrain.shape)
print(ytrain.shape)

(140123, 432)
(140123, 58)


In [23]:
df_xtrain = pd.DataFrame(xtrain, columns = [i for i in range(432)])

In [24]:
df_ytrain = pd.DataFrame(ytrain, columns = [f'R{i}' for i in range(58)])

In [25]:
df_series = pd.concat([df_xtrain, df_ytrain], axis = 1)

In [26]:
df_series.shape

(140123, 490)

In [27]:
xtrain.reshape(140123, 432, 1)
ytrain.reshape(140123, 58, 1)

array([[[-0.25800198],
        [-0.2401932 ],
        [-0.2401932 ],
        ...,
        [-0.04429658],
        [-0.06210537],
        [-0.07991415]],

       [[-0.2401932 ],
        [-0.2401932 ],
        [-0.2401932 ],
        ...,
        [-0.06210537],
        [-0.07991415],
        [-0.09772293]],

       [[-0.2401932 ],
        [-0.2401932 ],
        [-0.25800198],
        ...,
        [-0.07991415],
        [-0.09772293],
        [-0.09772293]],

       ...,

       [[ 0.04474733],
        [ 0.00912977],
        [-0.0264878 ],
        ...,
        [ 1.82562568],
        [ 1.87905203],
        [ 1.93247838]],

       [[ 0.00912977],
        [-0.0264878 ],
        [-0.06210537],
        ...,
        [ 1.87905203],
        [ 1.93247838],
        [ 1.93247838]],

       [[-0.0264878 ],
        [-0.06210537],
        [-0.07991415],
        ...,
        [ 1.93247838],
        [ 1.93247838],
        [ 1.87905203]]])

In [28]:
# label
labels_col = [f'R{i}' for i in range(58)]
labels = df_series[labels_col]
series = df_series.drop(labels_col, axis=1)
labels.values.shape

(140123, 58)

In [29]:
# x_train, x_val, y_train, y_val 분류
x_train, x_val, y_train, y_val = train_test_split(series, labels.values, test_size = 0.4, random_state = 0)

In [30]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,422,423,424,425,426,427,428,429,430,431
133922,-0.453899,-0.453899,-0.453899,-0.471707,-0.489516,-0.525134,-0.542943,-0.560751,-0.578560,-0.578560,...,-0.008679,0.044747,0.044747,0.062556,0.062556,-0.008679,-0.062105,-0.115532,-0.168958,-0.204576
32870,-0.418281,-0.418281,-0.436090,-0.453899,-0.453899,-0.471707,-0.471707,-0.471707,-0.489516,-0.489516,...,-0.489516,-0.489516,-0.489516,-0.489516,-0.507325,-0.525134,-0.525134,-0.525134,-0.542943,-0.542943
53652,-0.382663,-0.400472,-0.400472,-0.418281,-0.418281,-0.453899,-0.471707,-0.471707,-0.436090,-0.436090,...,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839,-0.738839
2311,-0.489516,-0.489516,-0.489516,-0.489516,-0.489516,-0.489516,-0.489516,-0.489516,-0.507325,-0.525134,...,-0.364855,-0.364855,-0.382663,-0.382663,-0.400472,-0.400472,-0.418281,-0.436090,-0.453899,-0.453899
132218,-0.667604,-0.667604,-0.685413,-0.685413,-0.685413,-0.685413,-0.703222,-0.703222,-0.721030,-0.721030,...,-0.560751,-0.560751,-0.560751,-0.631986,-0.631986,-0.631986,-0.631986,-0.631986,-0.631986,-0.649795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,0.026939,0.026939,0.026939,0.026939,0.026939,0.044747,0.044747,0.062556,0.062556,0.062556,...,0.294070,0.294070,0.294070,0.258453,0.222835,0.205026,0.187218,0.187218,0.151600,0.133791
97639,1.148892,1.042039,0.935187,0.810525,0.721481,0.650246,0.596820,0.543393,0.489967,0.436541,...,-0.970353,-0.970353,-0.970353,-0.970353,-0.970353,-0.988162,-0.970353,-0.970353,-0.988162,-0.970353
95939,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,-0.774457,...,-0.970353,-0.952545,-0.952545,-0.970353,-0.952545,-0.934736,-0.899118,-0.881309,-0.863501,-0.845692
117952,0.400923,0.383114,0.347497,0.329688,0.311879,0.294070,0.276262,0.258453,0.240644,0.222835,...,1.220127,1.220127,1.220127,1.148892,1.059848,0.952995,0.881760,0.810525,0.774907,0.739290


In [31]:
# xtest = 

In [32]:
# 모델링 및 모델 학습

if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history


#He_normal
He_normal = keras.initializers.he_normal(seed=None)

model = Sequential()
model.add(Conv1D(filters = 8, input_shape = (w, 1), kernel_size = 2, padding='causal', dilation_rate=2, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  #0.2~0.5  0.8까지

model.add(Conv1D(filters = 16, kernel_size = 2, padding='causal', dilation_rate=4, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 32, kernel_size = 1))

model.add(Flatten())   #model.add(GlobalAveragePooling1D(pool_size=2))

model.add(Dense(50, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(58))


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

opt = tf.keras.optimizers.Adam(lr=0.005) #기본값 0.001
model.compile(optimizer=opt, loss=root_mean_squared_error, metrics =["accuracy"])
model.summary()



callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]



history = model.fit(xtrain, ytrain, epochs=300, validation_data = (x_val, y_val), verbose=1, callbacks=callbacks, batch_size = 3742) #batch size..?


keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

keras_model_best.evaluate(x_val,y_val)

# y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 432, 8)            24        
                                                                 
 batch_normalization (BatchN  (None, 432, 8)           32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 432, 8)            0         
                                                                 
 dropout (Dropout)           (None, 432, 8)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 432, 16)           272       
                                                                 
 batch_normalization_1 (Batc  (None, 432, 16)          64        
 hNormalization)                                        

In [33]:
x_test = x['잠수교_수위'][-432:]

In [35]:
x_test = np.array(x_test).reshape((1, 432, 1))

In [36]:
x_test.shape

(1, 432, 1)

In [37]:
y_predict = keras_model_best.predict(x_test)

In [38]:
y_predict.shape

(1, 58)

In [39]:
pd.Series(list(y_predict))

0    [326.47412, 327.77902, 328.56757, 329.5968, 32...
dtype: object

In [40]:
수위.loc[consecutive, '잠수교_수위'] = y_predict

In [41]:
수위.loc[consecutive, '잠수교_수위']

140613    326.474121
140614    327.779022
140615    328.567566
140616    329.596802
140617    329.462372
140618    331.002747
140619    328.016205
140620    328.526154
140621    327.953033
140622    326.275604
140623    323.677887
140624    322.795593
140625    318.366272
140626    318.252075
140627    316.452179
140628    311.880798
140629    311.123505
140630    310.765106
140631    308.705078
140632    306.455811
140633    304.086365
140634    303.012360
140635    300.360352
140636    297.729187
140637    297.314331
140638    296.220337
140639    293.730042
140640    292.149841
140641    290.073975
140642    288.613129
140643    286.960602
140644    286.372772
140645    283.204163
140646    284.362152
140647    280.905853
140648    282.342224
140649    282.188477
140650    281.102112
140651    281.411713
140652    280.891418
140653    278.977051
140654    276.807770
140655    278.005432
140656    276.592621
140657    274.383270
140658    274.648193
140659    273.379974
140660    270

In [42]:
# unconsecutive 부분 보간

unconsecutive2 = [unconsecutive-1, unconsecutive, unconsecutive+1]
수위.loc[unconsecutive2, '잠수교_수위'] = 수위.loc[unconsecutive2, '잠수교_수위'].interpolate()

In [43]:
수위['잠수교_수위'].isnull().sum()

0

In [45]:
수위['잠수교_수위'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/CNN_전처리/잠수교_수위.csv', index = False, encoding = 'cp949')